In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
df = pd.read_csv('50_Startups.csv')
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [ ]:
df.isnull().sum()

In [ ]:
sns.regplot(df, x='R&D Spend' ,y= 'Profit')

In [ ]:
sns.regplot(df,x='Marketing Spend',y='Profit')

In [ ]:
sns.pairplot(df,kind='reg')

In [ ]:
sns.barplot(df,x="State",y='Profit')

In [ ]:
X= df.drop(columns=['Profit'])
y= df['Profit']

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
Xtrain , Xtest , ytrain ,ytest=train_test_split(X,y,test_size=0.2,random_state=0)

Column Selection

In [ ]:
cat_cols=X.select_dtypes(include=['object']).columns
num_cols=X.select_dtypes(include=['number']).columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


PipeLine

In [ ]:
cat_pipe=Pipeline(
    steps=[
        ('encoder',OneHotEncoder(drop='first'))
    ]
)


In [ ]:
num_pipe=Pipeline(
    steps=[
        ('scaler',StandardScaler())
    ]
)

In [ ]:
preprocessor= ColumnTransformer(transformers=[
    ('cat', cat_pipe, cat_cols),
    ('num',num_pipe,num_cols)
])

In [ ]:
model=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('lr',LinearRegression())
])

In [ ]:
model.fit(Xtrain,ytrain)

In [ ]:

from joblib import dump, load #dump is used to save the model and load is used to load the model 
with open('models/multiple_linear_regression.pkl','wb')as f:
    dump(model, f) # dumping the model into the file
    print("Model dumped successfully")

In [ ]:
! pip install gradio

In [ ]:
import gradio as gr
import pandas as pd
# make a single record DataFrame
def prediction(rnd, admin,marketing, state):
    df=pd.DataFrame({
        'R&D Spend':[rnd],
        'Administration':[admin],
        'Marketing Spend':[marketing],
        'State':[state],
    })
    #load joblib model file
    with open('models/multiple_linear_regression.pkl','rb')as f:
        model=load(f)


    #predict the profit
    profit =model.predict(df)# return the numpy array
    return profit[0]#return the profit value
ui= gr.Interface(
    fn=prediction,
    inputs=['number','number','number','text'],
    outputs=gr.Text(),
    title='50 startups profut prediction',
    examples=[[165349,136897,471784,'New York']]
)
ui.launch()